In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import time

import sys
sys.path.append("../..")
from utils import utils, pos_utils, model_utils
from zero_shot import zero_shot

### PoS - German

In [4]:
tagset = pos_utils.get_ud_tags()
params = {
    "data_path": "../../data/ud/",
    "results_path": "results/",
    "short_model_name": None,
    "task": "pos",
    "checkpoint_dir": None,
    "experiment": "acl",
    "max_length": 256,
    "batch_size": 64,
    "num_labels": len(tagset),
    "tagset": tagset
}

In [5]:
testing_langs = utils.get_langs("acl")
train_lang = "German"
train_lang_code = "de"
sizes = ["10k", "40k", "80k", "150k"]
short_model_names = ["mbert", "xlm-roberta"]
filepath = "results/results_de_large.xlsx"
df = pd.read_excel(filepath)
results = df.to_dict(orient="records")

for short_model_name in short_model_names:
    # Load model
    print("Now evaluating", short_model_name)
    model_name = model_utils.get_full_model_names(short_model_name)[0]
    params["short_model_name"] = short_model_name
    tester = zero_shot.Tester(**params)
    tester.setup_model()
    tester.current_model_lang = train_lang_code
    
    for size in tqdm(sizes, leave=False):
        print("Size:", size)
        tester.model.load_weights(
            "E:/TFM_CCIL/checkpoints_experimental/{}_{}/{}_pos.hdf5".format(train_lang_code, size, model_name)
        )
        size = int(size.replace("k", "000"))

        for test_lang in tqdm(testing_langs, leave=False):
            record = {"Train-Language": train_lang,
                      "Test-Language": test_lang,
                      "Model": short_model_name,
                      "Size": size}
            if record in df.iloc[:, :4].to_dict(orient="records"):
                print("Already tested", test_lang)
                time.sleep(0.1)
            else:
                test_lang_code = utils.name_to_code[test_lang]
                score = tester.test_on_lang(train_lang_code, test_lang_code, verbose=0)[0]
                results.append({**record, "Score": score})
                pd.DataFrame(results).to_excel(filepath, index=False)

Now evaluating mbert


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertForTokenClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['dropout_37', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Size: 10k


Size: 40k


Size: 80k


...

Some weights of the model checkpoint at jplu/tf-xlm-roberta-base were not used when initializing TFXLMRobertaForTokenClassification: ['lm_head']
- This IS expected if you are initializing TFXLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFXLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFXLMRobertaForTokenClassification were not initialized from the model checkpoint at jplu/tf-xlm-roberta-base and are newly initialized: ['classifier', 'dropout_76']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Size: 10k


Size: 40k


Size: 80k


Size: 150k


German on German PUD

In [2]:
tagset = pos_utils.get_ud_tags()
params = {
    "data_path": "../../data/ud/",
    "results_path": None,
    "short_model_name": None,
    "task": "pos",
    "checkpoint_dir": None,
    "experiment": "acl",
    "max_length": 256,
    "batch_size": 64,
    "num_labels": len(tagset),
    "tagset": tagset
}

In [3]:
train_lang = "German"
train_lang_code = "de"
test_lang = "German"
sizes = ["10k", "40k", "80k", "150k"]
short_model_names = ["mbert", "xlm-roberta"]
filepath = "results/results_de_pud.xlsx"
df = pd.read_excel(filepath)
results = df.to_dict(orient="records")

for short_model_name in short_model_names:
    # Load model
    print("Now evaluating", short_model_name)
    model_name = model_utils.get_full_model_names(short_model_name)[0]
    params["short_model_name"] = short_model_name
    tester = zero_shot.Tester(**params)
    tester.setup_model()
    tester.current_model_lang = train_lang_code
    data, dataset, batches = tester.load_data("extra_data/")
    
    for size in tqdm(sizes, leave=False):
        print("Size:", size)
        tester.model.load_weights(
            "E:/TFM_CCIL/checkpoints_experimental/{}_{}/{}_pos.hdf5".format(train_lang_code, size, model_name)
        )
        record = {"Train-Language": train_lang,
                  "Test-Language": test_lang,
                  "Model": short_model_name,
                  "Size": size}
        if record in df.iloc[:, :4].to_dict(orient="records"):
            print("Already tested", test_lang)
            time.sleep(0.1)
        else:
            test_lang_code = utils.name_to_code[test_lang]
            preds = tester.model.predict(dataset, steps=batches, verbose=1)
            score = tester.get_scores_pos(preds, data)[0]
            results.append({**record, "Score": score})
            pd.DataFrame(results).to_excel(filepath, index=False)

Now evaluating mbert


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertForTokenClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier', 'dropout_37']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Size: 10k
16/16 [==============================] - 16s 971ms/step
Size: 40k
16/16 [==============================] - 16s 978ms/step
Size: 80k
16/16 [==============================] - 16s 977ms/step
Size: 150k
16/16 [==============================] - 16s 995ms/step
Now evaluating xlm-roberta


Some weights of the model checkpoint at jplu/tf-xlm-roberta-base were not used when initializing TFXLMRobertaForTokenClassification: ['lm_head']
- This IS expected if you are initializing TFXLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFXLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFXLMRobertaForTokenClassification were not initialized from the model checkpoint at jplu/tf-xlm-roberta-base and are newly initialized: ['classifier', 'dropout_76']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Size: 10k
16/16 [==============================] - 16s 984ms/step
Size: 40k
16/16 [==============================] - 16s 1s/step
Size: 80k
16/16 [==============================] - 16s 1s/step
Size: 150k
16/16 [==============================] - 17s 1s/step


### Sentiment - Korean

In [2]:
params = {
    "data_path": "../../data/sentiment/",
    "results_path": "results/",
    "short_model_name": None,
    "task": "sentiment",
    "checkpoint_dir": None,
    "experiment": "acl",
    "max_length": 256,
    "batch_size": 64,
    "num_labels": 2
}

In [3]:
testing_langs = utils.get_langs("acl")
train_lang = "Korean"
train_lang_code = "ko"
sizes = ["10k", "40k", "80k", "140k"]
short_model_names = ["mbert", "xlm-roberta"]
filepath = "results/results_ko_large.xlsx"
df = pd.read_excel(filepath)
results = df.to_dict(orient="records")

for short_model_name in short_model_names:
    # Load model
    print("Now evaluating", short_model_name)
    model_name = model_utils.get_full_model_names(short_model_name)[0]
    params["short_model_name"] = short_model_name
    tester = zero_shot.Tester(**params)
    tester.setup_model()
    tester.current_model_lang = train_lang_code
        
    for size in tqdm(sizes, leave=False):
        print("Size:", size)
        tester.model.load_weights(
            "E:/TFM_CCIL/checkpoints_experimental/{}_{}/{}_sentiment.hdf5".format(train_lang_code, size, model_name)
        )
        size = int(size.replace("k", "000"))

        for test_lang in tqdm(testing_langs, leave=False):
            record = {"Train-Language": train_lang,
                      "Test-Language": test_lang,
                      "Model": short_model_name,
                      "Size": size}
            if record in df.iloc[:, :4].to_dict(orient="records"):
                print("Already tested", test_lang)
                time.sleep(0.1)
            else:
                test_lang_code = utils.name_to_code[test_lang]
                score = tester.test_on_lang(train_lang_code, test_lang_code, verbose=0)[-1] # Get F1 only
                results.append({**record, "Score": score})
                pd.DataFrame(results).to_excel(filepath, index=False)

Now evaluating mbert


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertForSequenceClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['dropout_37', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Size: 10k


e:\anaconda\envs\tfm_ccil\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Size: 40k


...

Some weights of the model checkpoint at jplu/tf-xlm-roberta-base were not used when initializing TFXLMRobertaForSequenceClassification: ['lm_head']
- This IS expected if you are initializing TFXLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFXLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFXLMRobertaForSequenceClassification were not initialized from the model checkpoint at jplu/tf-xlm-roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Size: 10k


Size: 40k


Size: 80k


Size: 140k
